**Curso de Inteligencia Artificial y Aprendizaje Profundo**

# Auto codificadores Variacionales (VAE)

##  Autores

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 

## Contenido

* [Introducción](#Introducción)
* [Inferencia Variacional](#Inferencia-Variacional)
 * [Planteamiento del problema variacional](#Planteamiento-del-problema-variacional)
 * [Aproximación Variacional](#Aproximación-Variacional)
 * [Divergencia Kullback-Leibler](#Divergencia-Kullback-Leibler)
 * [Optimización](#Optimización)
* [Implementación de un Autoencoder Variacional](#Implementación-de-un-Autoencoder-Variacional)
* [Ejemplo. Autoencoder Variacional Dense MNIST](VAE_Example_Dense_mnist.ipynb#Contenido)

## Introducción

Los autocodificadores variacionales constituyen una herramienta para problemas en los que se supone que existe una variable latente que genera los datos de entrada.

Para fijar ideas, supongamos que se tiene el resultado de una prueba académica aplicada a una muestra de personas. Más aún, supongamos que la codificación es binaria, en donde 1 (uno) codifica una respuesta correcta y 0 (cero) una respuesta incorrecta. 

Observe que la representación vectorial de la base de datos *mnist*, puede considerarse similar si cada pixel se codifica como cero o uno únicamente.

Entonces a la entrada se tienen vectores binarios  $\mathbf{x}$. de tamaño fijo, digamos $d=100$. Se busca entonces una respresentación de estos vectores en un espacio Euclideano de dimensión $m$, en donde $m<<d$. Digamos $m=5$. 

Los expertos utilizan distintas técnicas como por ejemplo el empleo del análisis de componentes principales (ACP) o la $q$-dimensión, para detectar la dimensión $m$.

La siguiente gráfica ilustra la técnica para el ejmeplo de datos de una prueba académica. 




<figure>
<center>
<img src="../Imagenes/autoencoder_intro.png" width="500" height="500" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Autoencoder Variacional</p>
</figcaption>
</figure>

En el espacio de entrada se tiene vectores de respuestas, Cada uno de ellos denotado por $\mathbf{x}$. 


Matemáticamente, estos objetos estan en el espacio $ A= \{0,1\}^d$. Es decir, sus elementos son vectores de tamaño $d$ y las componentes de cada vector son unos y ceros. La respuestas en la prueba de una  determinada persona se codifican en un vector $\mathbf{x}\in A$. Sobre $A$ se supone definida una medida de probabilidad que genéricamente denotaremos $P_{\theta}(\mathbf{x})$. Entonces se asume que  $\mathbf{x}$ es una muestra generada por dicha medida de probabilidad y se escribe $\mathbf{x}\sim P_{\theta}$. El símbolo $\theta$ refrefeiereiere a los parámetros (desconocidos) de la medida de probabilidad.

Ahora bien, se supone que la capacidad de una persona para resolver la prueba puede ser representada por un vector aleatorio $\mathbf{z} \in \mathbb{R}^m$. En principio, la teoría psicométrica está orientada a predecir un valor para $\mathbf{z}$, que en una prueba de Estado correponde a una calificación.

No haremos eso aquí, pero si generaremos muestras que correspondería a distintos valores que podría representar la capacidad de la persona. Estos valores se llaman muestras de las variables latentes. 


Si ahora se considera a un individuo en particular que  tiene capacidad $\mathbf{z}$ para resolver la prueba y obtiene el vector de respuestas $\mathbf{x}$, claramente existe una relación entre esos dos vectores que desde el punto de vista probabilístico se escribe de la siguiente forma:

1. El vector de respuestas $\mathbf{x}$ es generado a partir de la  distribución condicional $P(\mathbf{x}|\mathbf{z})$. En términos estadísticos $P(\mathbf{x}|\mathbf{z})$ es conocida excepto por sus parámetros. Estimar los parámetros de esta distribución será el problema central de una **autoencoder variacional** y es la parte que corresponde al decodificador. Aquí por lo general se introduce un modelo estadístico, por ejemplo un modelo de Bernoulli, y se estiman sus parámetros, haciendo uso de la entropía cruzada como función de pérdida.

2. La variable latente $\mathbf{z}$ es generada por la distribución condicional $Q(\mathbf{z}|\mathbf{x})$. Esta es siempre desconocida y justamente el problema de la inferencia variacional es encontrar una distribución simple, Gaussiana por ejemplo, que aproxime lo mejor posible a $Q(\mathbf{z}|\mathbf{x})$. Desde el punto de vista del autoencoder variacional, esta parte corresponde al codificador.


En la siguiente sección se muestra el desarrollo teórico del problema de inferencia variacional. Solamente para lectores con algún conocimiento matemático. Puede omitir sin problema. 

* [Ir a la implementación](#Implementación-de-un-Autoencoder-Variacional) 
* [Regresar al inicio](#Contenido)

## Inferencia Variacional

El propósito de la inferencia variacional es aproximar una densidad haciendo una paso intermedio por un espacio de variables latentes.

El proceso  puede ser imaginado de una forma análoga a la construcción de un codificador en variables latentes.

### Planteamiento del problema variacional

Supongamos que $p_{\theta}(\mathbf{x})$ es la densidad asociada a un vector aleatorio de respuesta. El problema estadístico en principio es estimar el parámetro $\theta$ que indexa a la distribución.

Si se asume que $\mathbf{z}$ es el vector latente asociado a $\mathbf{x}$  se tiene que

$$
P_{\theta}(\mathbf{x})= \int P_{\theta}(\mathbf{z},\mathbf{x})d\mathbf{z} = \int P_{\theta}(\mathbf{z}|\mathbf{x})P(\mathbf{x})d\mathbf{z}.
$$

Un esquema de muestreo para $\mathbf{z}$ se deriva de esta ecuación:

1. Dada una muestra $\mathbf{x} \sim P_{\theta}(x)$, es decir una muestra en la entrada de la red.
2. Se obtiene una muestra $\mathbf{z}\sim P_{\theta}(\mathbf{z}|\mathbf{x})$.


El problema es que en general $P_{\theta}(\mathbf{z}|\mathbf{x})$ es intratable. Lo que vamos a hacer para obtener la muestras $\mathbf{z}$ es obtener una densidad aproximada $Q_{\phi}(\mathbf{z}|\mathbf{x})$ en una familia conocida.

En esta lección lo que haremos es proponer una aproximación del tipo $Q_{\phi}(\mathbf{z}|\mathbf{x})= \mathcal{N}_m(\boldsymbol{\mu}(\mathbf{x}),\text{diag}(\boldsymbol{\sigma}(\mathbf{x})^2)) $.

El problema en la inferencia variacional es jusramente encontrar una buen aproximante $Q_{\phi}(\mathbf{z}|\mathbf{x})$, talque $Q_{\phi}(\mathbf{z}|\mathbf{x}) \approx P_{\theta}(\mathbf{z}|\mathbf{x})$.

 
 

### Aproximación Variacional

Con el propósito de convertir $p_{\theta}(\mathbf{z}|\mathbf{x})$ en una función de densidad tratable la solución propuesta desde la inferencia variacional es la introducción de una densidad aproximada $Q_{\phi}(\mathbf{z}|\mathbf{x})$ de tal manera que

$$
Q_{\phi}(\mathbf{z}|\mathbf{x}) \approx P_{\theta}(\mathbf{z}|\mathbf{x}).
$$

La densidad $Q_{\phi}(\mathbf{z}|\mathbf{x})$ se escoge en una familia de distribuciones tratables indexadas por $\phi$. Es común escoger $Q_{\phi}(\mathbf{z}|\mathbf{x})$ en la familia normal multivariada. Eso haremos en esta lección. 

Entonces para cada $\mathbf{x}$ tendremos que

$$
Q_{\phi}(\mathbf{z}|\mathbf{x}) = \mathcal{N}(\mathbf{z}; \boldsymbol{\mu}(\mathbf{x}), \text{diag}(\boldsymbol{\sigma}(\mathbf{x})^2)
$$

$\boldsymbol{\mu}(\mathbf{x})$ es el vector de medias (condicionadas por $\mathbf{x}$) y $\boldsymbol{\sigma}(\mathbf{x})$ es un vector de desviaciones estándar. Como la matriz de covarianza es diagonal, se está asumiendo que las componentes del vector $\mathbf{z}$ son condicionalmente independientes, dado el vector  $\mathbf{x}$.

### Divergencia Kullback-Leibler (KL)

Una vez se ha definido la familia de distribuciones a partir de la cual se obtendrá la aproximación $Q_{\phi}(\mathbf{z}|\mathbf{x})$ el siguiente paso es decidir como medir la proximidad o la discrepancia de la densidad aproximante con la densidad original. La solución sugerida desde la inferencia variacional es usar la divergencia KL, la cual se define por

$$
D_{KL}(Q_{\phi}\left(\mathbf{z}|\mathbf{x})|| p_{\theta}(\mathbf{z}|\mathbf{x})\right)  = \mathbb{E}_{\phi}(\log Q_{\phi}\left(\mathbf{z}|\mathbf{x}) - \log p_{\theta}(\mathbf{z}|\mathbf{x})\right)) = \int(\log Q_{\phi}\left(\mathbf{z}|\mathbf{x}) - \log p_{\theta}(\mathbf{z}|\mathbf{x})\right)Q_{\phi}(\mathbf{z}|\mathbf{x})dz.
$$

El símbolo $\mathbb{E}_{\phi}$ indica que la esperanza es con respecto a la densidad $Q_{\phi}\left(\mathbf{z}|\mathbf{x}\right)$.

### Cota inferior de la evidencia (ELBO)

El objetivo en la inferencia variacional es encontrar una densidad aproximante $Q_{\phi}(\mathbf{z}|\mathbf{x})$  para la densidad $p_{\theta}(\mathbf{z}|\mathbf{x})$ utilizando como métrica la divergencia KL, que por cierto no es una distancia, dado que no es simétrica.

A partir del teorema de Bayes se obtiene que 

$$
P_{\theta}(\mathbf{z}|\mathbf{x})= \frac{P_{\theta}(\mathbf{x}|\mathbf{z})P_{\theta}(\mathbf{z})}{P_{\theta}(\mathbf{x})}
$$

Por lo que la divergencia KL se transforma en

$$
D_{KL}(Q_{\phi}(\mathbf{z}|\mathbf{x})|| p_{\theta}(\mathbf{z}|\mathbf{x}))  = \mathbb{E}_{\phi}(\log Q_{\phi}\left(\mathbf{z}|\mathbf{x}) - \log p_{\theta}(\mathbf{x}|\mathbf{z})- \log P_{\theta}(\mathbf{z})\right)) + \log P_{\theta}(\mathbf{x}).
$$

De donde se obtiene que


$$
\log P_{\theta}(\mathbf{x}) - 
D_{KL}[Q_{\phi}(\mathbf{z}|\mathbf{x})|| p_{\theta}(\mathbf{z}|\mathbf{x})] = \mathbb{E}_{\phi}[\log p_{\theta}(\mathbf{x}|\mathbf{z})]-
D_{KL}[Q_{\phi}(\mathbf{z}|\mathbf{x})|| p_{\theta}(\mathbf{z})]
$$

Esta ecuación constituye el núcleo de la inferencia variacional. El lado izquierdo de la ecuación  contiene el término $P_{\theta}(\mathbf{x})$ que se busca maximizar menos el error de la aproximación medido por $D_{KL}[Q_{\phi}(\mathbf{z}|\mathbf{x})|| p_{\theta}(\mathbf{z}|\mathbf{x})]$ que se espera  sea aproximadamente cero. 


Se sabe que la divergencia KL siempre es positiva, por lo que la parte izquierda de la ecuación se denomina como la cota inferior de la evidencia (**ELBO**) del inglés *evidence lower bound*.

### Optimización

La ecuación clave de la inferencia variacional es dada por

$$
\begin{align}
\text{ELBO}  & = 
\log P_{\theta}(\mathbf{x}) - 
D_{KL}[Q_{\phi}(\mathbf{z}|\mathbf{x})|| p_{\theta}(\mathbf{z}|\mathbf{x})] \\
&= \mathbb{E}_{\phi}[\log p_{\theta}(\mathbf{x}|\mathbf{z})]-
D_{KL}[Q_{\phi}(\mathbf{z}|\mathbf{x})|| p_{\theta}(\mathbf{z})]
\end{align}
$$


El proceso de optimización se basa en la segunda parte ecuación, por lo que usaremos al definición

$$
\text{ELBO}  = \mathbb{E}_{\phi}[\log p_{\theta}(\mathbf{x}|\mathbf{z})]-
D_{KL}[Q_{\phi}(\mathbf{z}|\mathbf{x})|| p_{\theta}(\mathbf{z})]
$$


El término $\mathbb{E}_{\phi}[\log p_{\theta}(\mathbf{x}|\mathbf{z})]$ corresponde al modelo generativo en el problema. La interpretación estadística de este término es que el modelo generador toma muestras obtenidas de la salida del modelo latente $P_{\theta}(\mathbf{z}|\mathbf{x})$, el cual estamos aproximando con $Q_{\phi}(\mathbf{z}|\mathbf{x})$. Es decir, se genera una muestra $\mathbf{z} \sim Q_{\phi}(\mathbf{z}|\mathbf{x})$ y partir de esta se trata de reconstruir la entrada $\mathbf{x}$.

En el ejemplo propuesto, si se considera que se tiene vectores dicotómicos, se asume una distribución de Bernoulli para cada componente. Si las respuestas son condicionalmente independientes dado el vector $\mathbf{z}$ entonces la función de pérdida es la entropía cruzada binaria $\mathcal{L}_R$ dada por

$$
\mathcal{L}_R = - \frac{1}{d}\sum_{j=1}^d x_j \log p(\mathbf{w}_j'\mathbf{z} + b_j) + (1-x_j)\log(1-p(\mathbf{w}_j'\mathbf{z} + b_j))
$$


El segundo término $D_{KL}[Q_{\phi}(\mathbf{z}|\mathbf{x})|| p_{\theta}(\mathbf{z})]$ puede ser evaluado directamente. Como asumimos que $Q_{\phi}$ es una distribución Gaussiana y si se tiene en cuenta que típicamente $P_{\theta}(\mathbf{z})= P(\mathbf{z})=\mathcal{N}(\mathbf{0},\mathbf{I})$, se obtiene que 

$$
D_{KL}[Q_{\phi}(\mathbf{z}|\mathbf{x})|| p_{\theta}(\mathbf{z})]= \frac{1}{2} \sum_{j=1}^{d} (1+\log(\sigma_j)^2 - (\mu_j)^2-(\sigma_j)^2)
$$

Tanto $\mu_j$ como $\sigma_j$ son funciones de la entrada $\mathbf{x}$ ue se estiman en el modelo de inferencia.

Para minimizar $\mathcal{L}_{KL} =D_{KL}$, se requiere que $\mu_j\to 0$ y $\sigma_j\to 1$.

En resumen para el problema de inferencia variacional la función de pérdida es dada por

$$
\mathcal{L}_{VAE} = \mathcal{L}_{R} + \mathcal{L}_{KL}
$$

[Regresar al inicio](#Contenido)

### Implementación de un Autoencoder Variacional



Nuestro autocodificador (autoencoder) variacional (VAE) tendrá variables latentes gaussianas y una distribución posterior gaussiana   $q_{\phi}(\boldsymbol{z}|\boldsymbol{x})$  con una matriz de covarianza diagonal.

Recordemos que  un VAE de cuatro elementos escenciales:

1. Una  variable  latente $\boldsymbol{z}$ con distribución  $p(\boldsymbol{z})$  que en nuestro casoserá una variable  aleatoria Gaussiana con media cero y varianza 1 y que denotamos   $\epsilon$.
2. Un decodificador(decoder)  $p(\boldsymbol{x}|\boldsymbol{z})$  que mapea las  variables latentes  $\boldsymbol{z}$  a variables observables $\boldsymbol{x}$. En este ejemplo este codificador implementa un perceptron multicapa (MLP), es decir una red neuronal con una capa oculta.
3. Un codificador (encoder)  $q_{\phi}(\boldsymbol{z}|\boldsymbol{x})$  que mapea ejemplos de entrada al espacio latente. Como se stá cosntruyendo un autoencoder variacional se tiene que este mapeo se hace generando muestras aleatorias de distribciones Gaussianas con medias y varianzas que dependen de  la entrada:   $q_{\phi}(\boldsymbol{z}|\boldsymbol{x})=N(\boldsymbol{z},\boldsymbol{\mu}(x),\text{diag}(\boldsymbol{\sigma}^2(\boldsymbol{x})))$. 
4. Una función de costo que tiene dos términos: el  error de construcción que corresponde al modelo generativo implementado en el decoder y un término adicional de regularización que minimiza la divergencia KL. El error de reconstrucción es medido por el error cuadrático medio y la divergencia por el término
$-D_{KL}(q_{\phi}(\boldsymbol{z}|\boldsymbol{x})|p(\boldsymbol{z}))=\tfrac{1}{2}\sum_{j=1}^{J}(1+\log \boldsymbol{\sigma}^2_j(\boldsymbol{x})-\boldsymbol{\mu}^2_j(\boldsymbol{x})-\boldsymbol{\sigma}^2_j(\boldsymbol{x}))$.

### El truco de la reparametrización


Los gradientes de propagación hacia atrás no pueden pasar por el bloque de muestreo estocástico. Si bien está bien tener entradas estocásticas para redes neuronales, no es posible  pasar los gradientes por una capa estocástica. La solución a este problema es eliminar el proceso de muestreo como entrada, como se muestra en el lado derecho de la siguiente figura. 

Desde el punto de vista estadístico, esto es bastante simple. Se usa la representación estocástica clasica de una distribución normal multivariada asi:

1. Genera una muestra $\boldsymbol{\epsilon} \sim \mathcal{N}_m(\mathbf{0}, \mathbf{I})$.
2. Obtiene $\mathbf{z} =\boldsymbol{\mu} + \boldsymbol{\epsilon} \odot  \boldsymbol{\sigma}$.

En consecuencia $\mathbf{z}\sim \mathcal{N}_m(\boldsymbol{\mu},\text{diag}( \boldsymbol{\sigma}^2))$.

<figure>
<center>
<img src="../Imagenes/Probabilistico_intro.png" width="500" height="500" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Implementación de un  Autoencoder Variacional</p>
</figcaption>
</figure>

- [Regresar al inicio](#Contenido)
- [Ejemplo Auto Encoder Variacional para los datos de MNIST](VAE_Example_Dense_mnist.ipynb)